In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
# select the data
def select_data(data, start=2016, end=2024, columns=None):
    # convert the data to a pandas dataframe
    data['Date'] = pd.to_datetime(data['Date'])
    # create a mask for the data
    mask = (data['Date'].dt.year >= start) & (data['Date'].dt.year <= end)
    # select the data
    data = data.loc[mask]
    if columns is not None:
        data = data[columns]
    else:
        # drop the date column
        data = data.drop(columns='Date')
    return data

In [3]:
def prepare_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [4]:
def create_scaler(dataset):
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    return scaler, dataset 
def to_scalar(scaler, data):
    return scaler.transform(data)

In [5]:
def create_model(dataset, look_back=1, epochs=100, batch_size=1, verbose=1):
    X, Y = prepare_dataset(dataset, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    Y = np.reshape(Y, (Y.shape[0], Y.shape[1]))
    
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    #model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dense(X.shape[2]))  # Assuming single feature for simplicity
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    
    return model

In [7]:
# declare the path of the data
path_to_data = r"data.csv"

# load the data
# import pandas as pd
data = pd.read_csv(path_to_data)

col=['Rice', 'Wheat', 'Atta (Wheat)', 'Gram Dal', 'Tur/Arhar Dal']

# select the desired data
req_data = select_data(data=data, start=2024, end=2024, columns=col)
print(req_data.shape)

(249, 5)


In [9]:
# in last
col=['Date','Rice', 'Wheat', 'Atta (Wheat)', 'Gram Dal', 'Tur/Arhar Dal']
inp_data = select_data(data=data, start=2024, end=2024, columns=col)

In [10]:
print(inp_data.head())

           Date   Rice  Wheat  Atta (Wheat)  Gram Dal  Tur/Arhar Dal
2922 2024-01-01  59.04  44.45         54.08     86.88         171.79
2923 2024-01-02  58.89  44.91         53.22     86.83         171.58
2924 2024-01-03  58.74  45.12         52.94     86.80         170.77
2925 2024-01-04  59.44  45.00         53.40     86.37         170.26
2926 2024-01-05  60.03  44.69         53.00     86.88         170.38


In [11]:
# split the data into training and testing
train_size = int(len(req_data) * 0.80)
test_size = len(req_data) - train_size
train, test = req_data[0:train_size], req_data[train_size:len(req_data)]

In [12]:
# scale the data
scaler, train_scaler = create_scaler(train) # scaled_data is a numpy array

In [14]:
# create the model
myModel = create_model(train_scaler, look_back=1, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 999us/step - loss: 0.1507
Epoch 2/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0277
Epoch 3/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0185
Epoch 4/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0176
Epoch 5/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0188
Epoch 6/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0175
Epoch 7/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0172
Epoch 8/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0173
Epoch 9/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0175
Epoch 10/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0154
Epoch 11/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0157
Epoch 12/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0159  
Epoch 13/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0191
Epoch 14/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0154
Epoch 15/100
198/198 ━━━━━━━━━━━━━━━━━━

In [15]:
# prepare test data
# convert to scaler
test_scalar = scaler.transform(test)
# split features and outcome
x_test, y_test = prepare_dataset(test_scalar, look_back=1)

In [16]:
x_train, y_train = prepare_dataset(train_scaler, look_back=1)

In [17]:
# predict the results
test_pred_scalar = myModel.predict(x_test)
train_pred_scalar = myModel.predict(x_train)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [22]:
# invert the predictions
test_pred = scaler.inverse_transform(test_pred_scalar)
train_pred = scaler.inverse_transform(train_pred_scalar)

# invert the actual values
y_test = scaler.inverse_transform(y_test)
y_train = scaler.inverse_transform(y_train)

In [23]:
# print the shape of the data
print(train_pred.shape)
print(y_train.shape)
print(test_pred.shape)
print(y_test.shape)

(198, 5)
(198, 5)
(49, 5)
(49, 5)


In [24]:
# print the results dataframes first 5 rows
print(pd.DataFrame(train_pred).head())
print(pd.DataFrame(y_train).head())
print(pd.DataFrame(test_pred).head())
print(pd.DataFrame(y_test).head())

           0          1          2          3           4
0  60.008518  44.941517  53.693611  87.093315  171.130646
1  59.489590  44.921036  53.275555  86.887817  170.716949
2  59.348980  44.957283  53.257317  86.863434  169.924377
3  59.881336  44.971832  53.426445  86.427040  169.805435
4  60.344288  45.001808  53.325615  86.898781  170.520142
       0      1      2      3       4
0  58.89  44.91  53.22  86.83  171.58
1  58.74  45.12  52.94  86.80  170.77
2  59.44  45.00  53.40  86.37  170.26
3  60.03  44.69  53.00  86.88  170.38
4  60.83  44.78  53.41  86.17  170.17
           0          1          2          3           4
0  61.445110  45.190136  54.819038  94.903206  181.273895
1  61.430550  45.339901  54.989235  94.340378  180.044891
2  61.450665  45.093678  54.790504  94.018402  180.003555
3  61.345203  44.793957  54.800812  94.173393  180.610153
4  61.426113  45.354733  54.964989  93.377441  179.012711
       0      1      2      3       4
0  61.50  45.87  55.35  94.97  180.03
